In [1]:
import sys, torch, transformers
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__, "| CUDA available?", torch.cuda.is_available())
print("Transformers:", transformers.__version__)


/home/Musaed/Documents/GitHub_My-Repos/Task2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python: 3.13.7
PyTorch: 2.8.0+cu128 | CUDA available? True
Transformers: 4.56.0


In [1]:
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

result = pipe(
    "I want to refinance my home loan",
    candidate_labels=["Mortgage Finance", "Personal Finance", "Real Estate"]
)
print(result)


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


{'sequence': 'I want to refinance my home loan', 'labels': ['Real Estate', 'Mortgage Finance', 'Personal Finance'], 'scores': [0.7073387503623962, 0.19481633603572845, 0.0978449210524559]}


In [2]:
from transformers import pipeline

# build pipeline (device=0 for GPU, -1 for CPU)
pipe = pipeline(
    task="zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli",
    device=0   # change to -1 if you want CPU only
)

def rerank_zero_shot(post_text, theme_texts, hypothesis_template="This post is about: {}"):
    """
    Returns list of (label, score) sorted by score descending.
    theme_texts: list of strings (theme descriptions in EN or AR)
    """
    out = pipe(
        sequences=post_text,
        candidate_labels=theme_texts,
        multi_label=True,
        hypothesis_template=hypothesis_template
    )
    # zip labels with scores, and sort highest first
    return sorted(zip(out["labels"], out["scores"]), key=lambda x: x[1], reverse=True)


# sample post in Arabic
post_ar = "أفكر في إعادة تمويل قرض المنزل بعد انخفاض أسعار الفائدة."

# english-only theme descriptions
themes = [
    "Mortgage Finance: home loans, refinancing, interest rates.",
    "Personal Finance: budgeting, credit cards, savings.",
    "Real Estate: buying/selling property, market news."
]

# bilingual (English + Arabic) theme descriptions
themes_bi = [
    "Mortgage Finance: القروض العقارية وإعادة التمويل وأسعار الفائدة.",
    "Personal Finance: الميزانية والبطاقات الائتمانية والادخار.",
    "Real Estate: السوق العقاري وبيع وشراء العقارات."
]

print("EN themes →")
print(rerank_zero_shot(post_ar, themes))

print("\nAR themes →")
print(rerank_zero_shot(post_ar, themes_bi, hypothesis_template="هذا النص يتحدث عن {}"))


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


EN themes →
[('Mortgage Finance: home loans, refinancing, interest rates.', 0.987980842590332), ('Personal Finance: budgeting, credit cards, savings.', 0.9428037405014038), ('Real Estate: buying/selling property, market news.', 0.7868875861167908)]

AR themes →
[('Mortgage Finance: القروض العقارية وإعادة التمويل وأسعار الفائدة.', 0.9959806799888611), ('Personal Finance: الميزانية والبطاقات الائتمانية والادخار.', 0.9238924384117126), ('Real Estate: السوق العقاري وبيع وشراء العقارات.', 0.8046794533729553)]
